<a href="https://colab.research.google.com/github/ArneHei/Backend_Mobility/blob/main/Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
global department_sequence_counters
department_sequence_counters = {}

def Transport_create(list_of_shipments_df):
    """
    Creates a Transport object from a DataFrame of shipments.
    The transport will have a unique ID: {Department}A-XXXX (sequential per department).

    Args:
        list_of_shipments_df (pd.DataFrame): A DataFrame containing shipment details.

    Returns:
        Transport: A newly created Transport object.

    Raises:
        ValueError: If the input DataFrame is empty or if sequential IDs for a department are exhausted.
    """
    if list_of_shipments_df.empty:
        raise ValueError("Input DataFrame is empty. Cannot create a Transport object.")

    # Get department from the first shipment
    department = list_of_shipments_df.iloc[0]['Department']

    # Get a list of Shipment_IDs for the Transport object
    Shipment_IDs = list_of_shipments_df['Shipment_ID'].tolist()

    # Calculate Pickup_date (earliest date) and Delivery_date (latest date)
    # Ensure dates are in datetime format for proper min/max calculation
    earliest_pickup_date = pd.to_datetime(list_of_shipments_df['Pickup_date']).min().date()
    latest_delivery_date = pd.to_datetime(list_of_shipments_df['Delivery_date']).max().date()

    # Calculate combined Weight, Volume, and Ldm
    total_weight = list_of_shipments_df['Weight'].sum()
    total_volume = list_of_shipments_df['Volume'].sum()
    total_ldm = list_of_shipments_df['Ldm'].sum()
    total_cost = list_of_shipments_df['Cost'].sum()

    # Collect stops from shipments, 'P' type before 'D' type
    pickup_stops = []
    delivery_stops = []
    for shipment_id in Shipment_IDs:
        # Assuming Shipment object has 'Pickup_location_ID' and 'Delivery_location_ID' or similar
        # that can be used to construct Stop IDs
        # Based on user input, Stop IDs are 'Shipment_ID_P' and 'Shipment_ID_D'
        pickup_stop_id = f"{shipment_id}_P"
        delivery_stop_id = f"{shipment_id}_D"

        pickup_stop_obj = Stop.get_by_id(pickup_stop_id)
        delivery_stop_obj = Stop.get_by_id(delivery_stop_id)

        if pickup_stop_obj:
            pickup_stops.append(pickup_stop_obj)
        else:
            print(f"Warning: Pickup Stop object for ID '{pickup_stop_id}' not found for shipment '{shipment_id}'.")
        if delivery_stop_obj:
            delivery_stops.append(delivery_stop_obj)
        else:
            print(f"Warning: Delivery Stop object for ID '{delivery_stop_id}' not found for shipment '{shipment_id}'.")

    transport_stops = pickup_stops + delivery_stops

    # Ensure the department has an entry in the sequence counter
    if department not in department_sequence_counters:
        department_sequence_counters[department] = 0 # Start from 0, so first increment makes it 1

    # Generate a unique sequential Transport ID
    Transport_ID = None
    max_sequence_value = 9999

    # Iterate to find the next available sequential ID for the department
    while True:
        department_sequence_counters[department] += 1
        sequence_num = department_sequence_counters[department]

        # Check if the sequence number has exceeded the allowed range (0001-9999)
        if sequence_num > max_sequence_value:
            raise ValueError(f"Exceeded maximum sequential IDs ({max_sequence_value}) for department '{department}'.")

        formatted_sequence = f"{sequence_num:04d}"
        proposed_id = f"TOUR01-{formatted_sequence}"

        # Check if this proposed ID is already in use in the Transport class's registry
        if Transport.get_by_id(proposed_id) is None:
            Transport_ID = proposed_id
            break

    # Create the Transport object using the generated unique ID and calculated dates and all new parameters
    new_transport = Transport(
        Transport_ID,
        department,
        Shipment_IDs,
        earliest_pickup_date,
        latest_delivery_date,
        total_weight,
        total_volume,
        total_ldm,
        total_cost # Cost for Transport object
    )

    # Update the Transport_ID for each shipment object
    new_transport.Stops = transport_stops
    for Shipment_ID in Shipment_IDs:
        shipment_obj = Shipment.get_by_id(Shipment_ID)
        if shipment_obj:
            shipment_obj.Transport = new_transport.Transport_ID

    print(f"Department set to {department} and Created Transport object with ID: {new_transport.Transport_ID}")
    return new_transport

In [ ]:
def Transport_add(transport_id, list_of_shipments_df):
    """
    Adds shipments from a DataFrame to an existing Transport object.
    Only shipments with 'Transport' set to None or an empty string are considered.

    Args:
        transport_id (str): The ID of the Transport object to add shipments to.
        list_of_shipments_df (pd.DataFrame): DataFrame containing shipment details.

    Returns:
        Transport: The updated Transport object, or None if not found.
    """
    transport_obj = Transport.get_by_id(transport_id)
    if not transport_obj:
        print(f"Error: Transport with ID '{transport_id}' not found.")
        return None

    # Filter shipments where 'Transport' is None or an empty string. This ensures we
    # only attempt to add shipments that appear unassigned in the input DataFrame.
    filtered_shipments_df = list_of_shipments_df[
        (list_of_shipments_df['Transport'].isna()) | (list_of_shipments_df['Transport'] == '')
    ].copy()

    if filtered_shipments_df.empty:
        print(f"No new shipments with Transport=None or empty string to add to {transport_id}.")
        return transport_obj

    # Initialize accumulation variables with current transport object's values
    current_weight = transport_obj.Weight
    current_volume = transport_obj.Volume
    current_ldm = transport_obj.Ldm
    current_cost = transport_obj.Cost

    added_shipment_ids = []
    new_pickup_stops = []
    new_delivery_stops = []

    for index, row in filtered_shipments_df.iterrows():
        shipment_id = row['Shipment_ID']
        shipment_obj = Shipment.get_by_id(shipment_id)

        # Ensure shipment_obj exists and is currently unassigned (None or empty string)
        # and is not already part of this specific transport object.
        if shipment_obj and \
           (shipment_obj.Transport is None or shipment_obj.Transport == '') and \
           shipment_id not in transport_obj.Shipments:
            # Add shipment ID to Transport object's list
            transport_obj.Shipments.append(shipment_id)

            # Update individual Shipment object's Transport attribute
            shipment_obj.Transport = transport_id

            # Accumulate totals
            current_weight += row['Weight']
            current_volume += row['Volume']
            current_ldm += row['Ldm']
            current_cost += row['Cost']

            # Collect new stops (now allowing duplicates as requested)
            pickup_stop_id = f"{shipment_id}_P"
            delivery_stop_id = f"{shipment_id}_D"

            pickup_stop_obj = Stop.get_by_id(pickup_stop_id)
            delivery_stop_obj = Stop.get_by_id(delivery_stop_id)

            if pickup_stop_obj:
                new_pickup_stops.append(pickup_stop_obj)
            else:
                print(f"Warning: Pickup Stop object for ID '{pickup_stop_id}' not found for shipment '{shipment_id}'.")
            if delivery_stop_obj:
                new_delivery_stops.append(delivery_stop_obj)
            else:
                print(f"Warning: Delivery Stop object for ID '{delivery_stop_id}' not found for shipment '{shipment_id}'.")

            added_shipment_ids.append(shipment_id)

    # Update the Transport object's attributes
    transport_obj.Weight = current_weight
    transport_obj.Volume = current_volume
    transport_obj.Ldm = current_ldm
    transport_obj.Cost = current_cost

    # Append new stops, pickup before delivery
    transport_obj.Stops.extend(new_pickup_stops)
    transport_obj.Stops.extend(new_delivery_stops)

    if added_shipment_ids:
        print(f"Successfully added shipments {added_shipment_ids} to Transport '{transport_obj.Transport_ID}'.")
        print(f"Updated Transport details: Weight={transport_obj.Weight}, Volume={transport_obj.Volume}, Ldm={transport_obj.Ldm}, Cost={transport_obj.Cost}, Stops={transport_obj.Stops}")
    else:
        print(f"No new shipments were added to Transport '{transport_obj.Transport_ID}'.")

    return transport_obj

In [ ]:
def Transport_remove(transport_id, shipment_ids):
    """
    Removes one or more shipments from a transport and updates the transport's metrics.

    Args:
        transport_id (str): The ID of the transport from which to remove the shipment(s).
        shipment_ids (list): A list of Shipment IDs to remove.

    Returns:
        Transport: The updated Transport object, or None if the transport was not found.
    """
    transport = Transport.get_by_id(transport_id)
    if not transport:
        print(f"Error: Transport with ID '{transport_id}' not found.")
        return None

    removed_count = 0
    for shipment_id in shipment_ids:
        if shipment_id not in transport.Shipments:
            print(f"Shipment '{shipment_id}' is not part of Transport '{transport_id}'. Skipping.")
            continue

        # Retrieve the corresponding Shipment object before removing its ID from transport.Shipments
        shipment_to_remove = Shipment.get_by_id(shipment_id)

        # 4a. Remove the shipment_id from the Transport object's Shipments list.
        transport.Shipments.remove(shipment_id)

        if not shipment_to_remove:
            print(f"Warning: Shipment object '{shipment_id}' not found in registry, but proceeding with transport update.")
        else:
            # 4c. Set its Transport attribute to None
            shipment_to_remove.Transport = None

            # 4d. Subtract the Weight, Volume, and Ldm of the removed Shipment object
            transport.Weight -= shipment_to_remove.Weight
            transport.Volume -= shipment_to_remove.Volume
            transport.Ldm -= shipment_to_remove.Ldm
            transport.Cost -= shipment_to_remove.Cost

            # Identify the specific Stop objects to be removed using their unique IDs
            stops_to_remove_objs = []
            pickup_stop_id = f"{shipment_id}_P"
            delivery_stop_id = f"{shipment_id}_D"

            pickup_stop_obj = Stop.get_by_id(pickup_stop_id)
            delivery_stop_obj = Stop.get_by_id(delivery_stop_id)

            if pickup_stop_obj:
                stops_to_remove_objs.append(pickup_stop_obj)
            if delivery_stop_obj:
                stops_to_remove_objs.append(delivery_stop_obj)

            # Create a new list of stops, excluding the identified Stop objects
            new_stops = []
            for stop in transport.Stops:
                # Check if the current stop object is one of the specific objects we want to remove
                if stop not in stops_to_remove_objs:
                    new_stops.append(stop)
            transport.Stops = new_stops

        removed_count += 1

    if removed_count > 0:
        # 4e. Print a confirmation message
        print(f"Successfully removed {removed_count} shipment(s) from Transport '{transport_id}'.")
        print(f"Updated Transport '{transport_id}' details: Weight={transport.Weight}, Volume={transport.Volume}, Ldm={transport.Ldm}, Cost={transport.Cost}, Stops={transport.Stops}")
    else:
        print(f"No shipments from the provided list were removed from Transport '{transport_id}'.")

    # 5. Return the updated Transport object.
    return transport

In [ ]:
def Transport_delete(transport_id):
    """
    Deletes a Transport object if it has no associated shipments.

    Args:
        transport_id (str): The ID of the Transport object to delete.

    Returns:
        bool: True if the transport was successfully deleted, False otherwise.
    """
    transport_obj = Transport.get_by_id(transport_id)
    if not transport_obj:
        print(f"Error: Transport with ID '{transport_id}' not found.")
        return False

    if transport_obj.Shipments:
        print(f"Error: Transport '{transport_id}' has {len(transport_obj.Shipments)} shipments and cannot be deleted.")
        return False

    # Assuming Transport class has a registry and a way to remove itself
    # This part depends on the exact implementation of the Transport class and its registry
    # A common pattern is to have a class-level registry (e.g., Transport._registry)
    # For now, let's assume a static method or direct dictionary access if it exists.
    if hasattr(Transport, '_registry') and transport_id in Transport._registry:
        del Transport._registry[transport_id]
        print(f"Transport '{transport_id}' successfully deleted.")
        return True
    elif hasattr(transport_obj, 'delete_from_registry'): # If there's an instance method to delete
        transport_obj.delete_from_registry()
        print(f"Transport '{transport_id}' successfully deleted.")
        return True
    else:
        print(f"Warning: Transport '{transport_id}' found but no clear mechanism to delete from registry. Please implement a deletion mechanism in the Transport class.")
        return False


In [ ]:
def Transport_assign(truck_obj, transport_obj):
    """
    Assigns a given Truck object to a Transport object.

    Args:
        truck_obj (Truck): The Truck object to assign.
        transport_obj (Transport): The Transport object to assign the truck to.

    Raises:
        ValueError: If the Transport already has a vehicle assigned.
    """
    if transport_obj.Vehicle != "":
        raise ValueError("Only one truck can be assigned to one Transport")

    # Update Transport object
    transport_obj.Vehicle = truck_obj.License_plate
    transport_obj.Driver = truck_obj.Driver
    if transport_obj.Trailer == "": # If Transport's trailer is empty, take the Truck's trailer
        transport_obj.Trailer = truck_obj.Trailer
    transport_obj.Haulier = truck_obj.Haulier

    # Update Truck object
    truck_obj.Transport = transport_obj.Transport_ID
    if transport_obj.Trailer != "": # If Transport's trailer is not empty (after potential update), Truck takes that value
        truck_obj.Trailer = transport_obj.Trailer

    print(f"Transport {transport_obj.Transport_ID} assigned to Truck {truck_obj.License_plate}.")
    print(f"Updated Transport details: Vehicle={transport_obj.Vehicle}, Driver={transport_obj.Driver}, Trailer={transport_obj.Trailer}, Haulier={transport_obj.Haulier}")
    print(f"Updated Truck details: Transport={truck_obj.Transport}, Trailer={truck_obj.Trailer}")
    return transport_obj, truck_obj

In [ ]:
def Transport_unassign(truck_obj, transport_obj):
    """
    Unassigns a Truck object from a Transport object.
    Reverts the changes made by Transport_assign, but the truck's trailer
    remains assigned to the truck. Transport trailer is set to "".

    Args:
        truck_obj (Truck): The Truck object to unassign.
        transport_obj (Transport): The Transport object to unassign the truck from.

    Raises:
        ValueError: If the Truck is not currently assigned to the given Transport.
    """
    if transport_obj.Vehicle != truck_obj.License_plate or truck_obj.Transport != transport_obj.Transport_ID:
        raise ValueError(f"Truck {truck_obj.License_plate} is not assigned to Transport {transport_obj.Transport_ID}.")

    # Update Transport object
    transport_obj.Vehicle = ""
    transport_obj.Driver = ""
    transport_obj.Haulier = ""
    transport_obj.Trailer = ""  # Reset Transport's trailer as per request

    # Update Truck object
    truck_obj.Transport = "" # Unassign transport from truck
    # truck_obj.Trailer is intentionally *not* changed, as per user's request

    print(f"Transport {transport_obj.Transport_ID} successfully unassigned from Truck {truck_obj.License_plate}.")
    print(f"Updated Transport details: Vehicle='{transport_obj.Vehicle}', Driver='{transport_obj.Driver}', Haulier='{transport_obj.Haulier}', Trailer='{transport_obj.Trailer}'")
    print(f"Updated Truck details: Transport='{truck_obj.Transport}', Trailer='{truck_obj.Trailer}' (unchanged from truck's original trailer)")
    return transport_obj, truck_obj